# URT [Chapter 1](http://underactuated.csail.mit.edu/underactuated.html?chapter=1) Notes

Wayne H Nixalo - 2018/2/14 00:53

---

### Example 1.1 Robot Manipulators

The equations of motion for the double pendulum system are simple to derive (see [Appendix](http://underactuated.csail.mit.edu/underactuated.html?chapter=multibody)) and take the form of standard "manipulator equations":

$$M(q)\ddot{q}+C(q,\dot{q})\dot{q} = \tau_g(q)+Bu$$

The inertial matrix $M(q)$ is always uniformly symmetric & positive definite and $\Rightarrow$ is invertible. Putting the equation into [Equation 1](http://underactuated.csail.mit.edu/underactuated.html?chapter=1#mjx-eqn-eqf1_plus_f2) form yields: 

$$\ddot{q}=M^{-1}(q)[\tau_g(q)+Bu-C(q,\dot{q}]\dot(q)$$

Because $\textbf{M}^{-1}(\textbf{q})$ is always full rank, the system described by the manipulator eqns is FA $\iff$ $\textbf{B}$ is full row rank. Here $\textbf{q}=[\theta_1,\theta_2]^T$ and $\textbf{u}=[\tau_1,\tau_2]^T$ (motor torques at joints), and $\textbf{B}=\textbf{I}_{2\times2}$

In [5]:
from pydrake.all import (BasicVector, DiagramBuilder, FloatingBaseType,
                         RigidBodyPlant, RigidBodyTree, Simulator)
from underactuated import (FindResource, PlanarRigidBodyVisualizer)

# Load the double pendulum from Universal Robot Description Format
tree = RigidBodyTree(FindResource("double_pendulum.urdf"),
                     FloatingBaseType.kFixed)

# Set up a block diagram with the robot (dynamics) and a visualization block.
builder = DiagramBuilder()
robot = builder.AddSystem(RigidBodyPlant(tree))
builder.ExportInput(robot.get_input_port(0))
visualizer = builder.AddSystem(PlanarRigidBodyVisualizer(tree,
                                                         xlim=[-2.8, 2.8],
                                                         ylim=[-2.8, 2.8]))
builder.Connect(robot.get_output_port(0), visualizer.get_input_port(0))
diagram = builder.Build()

# Set up a simulator to run this diagram
simulator = Simulator(diagram)
simulator.set_target_realtime_rate(1.0)
simulator.set_publish_every_time_step(False)

# Set the initial conditions
context = simulator.get_mutable_context()
context.FixInputPort(0, BasicVector([0., 0.]))  # Zero input torques
state = context.get_mutable_continuous_state_vector()
state.SetFromVector((1., 1., 0., 0.))  # (theta1, theta2, theta1dot, theta2dot)

# Simulate for 10 seconds
simulator.StepTo(10)

ImportError: No module named underactuated

We can also use [Drake](http://drake.mit.edu/) to evaluate manipulator eqns:

In [6]:
from pydrake.all import (FloatingBaseType, RigidBodyTree)
from underactuated import (FindResource, ManipulatorDynamics)

tree = RigidBodyTree(FindResource("double_pendulum.urdf"),
                     FloatingBaseType.kFixed)

q = (1., 1.)
v = (0.1, 0.1)
(M, Cv, tauG, B) = ManipulatorDynamics(tree, q, v)
print("M = " + str(M))
print("Cv = " + str(Cv))
print("tauG = " + str(tauG))
print("B = " + str(B))

ImportError: No module named underactuated

**NOTE**: hmm, looks like this stuff isn't fully implemented yet. I'll try it again later.